In [ ]:
from google.colab import files
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from sklearn import metrics
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math


In [ ]:
import os
os._exit(00)

Leemos el los datasets

In [ ]:
uploades=files.upload()

Saving sample_submission.csv to sample_submission (1).csv
Saving test.csv to test (1).csv
Saving train.csv to train (1).csv


In [ ]:
df_housestrain= pd.read_csv("train.csv")
df_housestest= pd.read_csv("test.csv")

In [ ]:
df_housestrain

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


Observo las columnas que tienen NaN values y lo comparo con train y test para ver que conclusiones puedo sacar. Es decir, si la columna la puedo deshechar por tener poca información, o bien, puedo llenar los valores con la media o una mediana.

In [ ]:
x=df_housestrain.isnull().sum()
x.to_csv('null_train.csv')
x=df_housestest.isnull().sum()
x.to_csv('null_test.csv')

Se observa que podemos dropear las columnas FireplaceQu, Alley, PoolQC, Fence y MiscFeature al tener más de 700 datos faltantes tanto en train como en test.

Por el momento, voy a dejar la columna LotFrontage que sería, despues de los cambios, la columna con más datos NaN con 227.

In [ ]:
columns_to_drop = ['FireplaceQu', 'Alley', 'PoolQC', 'Fence', 'MiscFeature','Id']

df_housestrain = df_housestrain.drop(columns_to_drop,axis=1)
df_housestest  = df_housestest.drop(columns_to_drop,axis=1)

Ahora tenemos las siguientes columnas con datos NaN:

**EN TRAIN:**
LotFrontage, MasVnrType, MasVnrArea,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2, Electrical,GarageType,GarageyrBlt, GarageFinish, GarageQual,GarageCond

**EN TEST** 
MSZoning, LotFrontage, Utilities,Exterior1st, Exterior2nd,MasVnrType,MasVnrArea,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,BsmtFullBath,BsmtHalfBath,KitchenQual,Functional,GarageType,GarageYrBlt,GarageFinish	,GarageCars	,GarageArea	,GarageQual	,GarageCond,SaleType

Vamos a rellenar (.fillna) cada uno de estos dataset con un promedio estadístico. Podemos considerar que para valores categoricos, lo más probable sea la moda y para valores numericos la media y la mediana. Para hacer el proyecto sencillo y no hacer dos casos distintos, vamos a realizar el siguiente esquema

*si es categorico -> rellenamos con moda* 

*si es numerico   -> rellenamos con media*

## **RELLENAMOS DATOS DE TRAIN**

In [ ]:
columnsfill_train_categorical = ["MasVnrType",'BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2',"Electrical","GarageType","GarageFinish",'GarageQual','GarageCond']
columnsfill_train_numerical   = ["LotFrontage","MasVnrArea","GarageYrBlt"]

for i in columnsfill_train_categorical:
  df_housestrain[i]=df_housestrain[i].fillna(df_housestrain[i].mode()[0])

for i in columnsfill_train_numerical:
  df_housestrain[i]=df_housestrain[i].fillna(df_housestrain[i].mean())

df_housestrain.to_csv('train_withoutnull.csv')
#Nos aseguramos con este excel que no hay ningun dato faltante y que todo se ha ejecutado correctamente
x=df_housestrain.isnull().sum()
x.to_csv('null_train_asegurar.csv')

## **RELLENAMOS DATOS DE TEST**

In [ ]:
columnsfill_test_categorical = ["MSZoning","Utilities",'Exterior1st','Exterior2nd','BsmtQual','BsmtCond','BsmtExposure',
                                'BsmtFinType1','BsmtFinType2','KitchenQual','Functional','GarageType','GarageFinish','GarageQual','SaleType','GarageCond']

columnsfill_test_numerical   = ["LotFrontage",'MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF',
                                'BsmtFullBath','BsmtHalfBath','GarageYrBlt','GarageCars','GarageArea']


for i in columnsfill_test_categorical:
  df_housestest[i]=df_housestest[i].fillna(df_housestest[i].mode()[0])

for i in columnsfill_test_numerical:
  df_housestest[i]=df_housestest[i].fillna(df_housestest[i].mean())




x=df_housestest.isnull().sum()
x.to_csv('null_test_asegurar.csv')

In [ ]:
df_housestest.to_csv('test_withoutnull.csv')
df_housestrain.to_csv('train_withoutnull.csv')

## Scaling Section.

Pasamos a predecir ya que tenemos bien hecho los datos. En primer lugar, escalamos los datos.


*Para datos categoricos -> One Hot Encoder*

*Para datos numericos -> Min Max Scaler*

In [ ]:
df_housestrain_data= pd.read_csv("train_withoutnull.csv")
df_housestest_data= pd.read_csv("test_withoutnull.csv")

In [ ]:
y_train=df_housestrain_data["SalePrice"]
x=df_housestrain_data.drop(["SalePrice"],axis=1)
#Las columnas categoricas de train son:
columns_categorical_train=["Utilities","MSZoning","Street","LotShape","LandContour","LotConfig","LandSlope","Neighborhood","Condition1",
                     "Condition2","BldgType","HouseStyle","RoofStyle","RoofMatl","Exterior1st","Exterior2nd","MasVnrType","ExterQual",
                     "ExterCond",'Foundation', 'BsmtQual','BsmtCond', 'BsmtExposure', 'BsmtFinType1','BsmtFinType2',
                     'Heating','HeatingQC', 'CentralAir', 'Electrical','KitchenQual','Functional', 'GarageType',"GarageFinish",
                     'GarageQual', 'GarageCond','PavedDrive','SaleType','SaleCondition']

columns_numerical_train=['MSSubClass','LotFrontage', 'LotArea','OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd','MasVnrArea',
                         'BsmtFinSF1','BsmtFinSF2','BsmtUnfSF', 'TotalBsmtSF','1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 
                         'BsmtHalfBath', 'FullBath','HalfBath', 'BedroomAbvGr', 'KitchenAbvGr','TotRmsAbvGrd','Fireplaces','GarageYrBlt',
                         'GarageCars', 'GarageArea','WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
                         'MoSold', 'YrSold']

Escalamos los datos de training

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
#Pasamos y_train a tipo numpy por problemas al pasarle la metrica y lo ponemos en forma de vector 1D
y_train = y_train.to_numpy(dtype ='float32')
y_train= y_train.reshape(-1, 1)

array([[208500.],
       [181500.],
       [223500.],
       ...,
       [266500.],
       [142125.],
       [147500.]], dtype=float32)

In [ ]:
#Divido el data set en dos partes: numerico y categorico
x_numerical=x.drop(columns_categorical_train, axis=1)
x_categorical=x.drop(columns_numerical_train, axis=1)

#Para valores categoricos:
x_categorical = pd.get_dummies(x_categorical, columns=columns_categorical_train)

#Para valores numericos:
#Escalado de X_train
scaler = StandardScaler().fit(x_numerical)
x_scaled = scaler.transform(x_numerical)
#Escalado de Y_train
scalery = StandardScaler().fit(y_train)
y_scaled = scalery.transform(y_train)

#Llamo a los datos en forma de Pandas ya que lo convierte automaticamente en numpy
x_categorical = pd.DataFrame(x_categorical, columns = x_categorical.columns)
x_scaled = pd.DataFrame(x_scaled, columns = x_numerical.columns)
#Juntamos los dos dataset escalados:
df_scaled_train = pd.concat( [x_categorical,x_scaled],axis=1)
#Eliminamos la columna generada por concatenar.
df_scaled_train = df_scaled_train.drop('Unnamed: 0',axis=1)

#Por lo que nuestro dataset de entrenamiento final es:

df_scaled_train.columns

Index(['Utilities_AllPub', 'Utilities_NoSeWa', 'MSZoning_C (all)',
       'MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL', 'MSZoning_RM',
       'Street_Grvl', 'Street_Pave', 'LotShape_IR1',
       ...
       'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold'],
      dtype='object', length=270)

Escalamos los datos de test

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
x=df_housestest_data
#Divido el data set en dos partes: numerico y categorico
x_numerical=x.drop(columns_categorical_train, axis=1)
x_categorical=x.drop(columns_numerical_train, axis=1)

#Para valores categoricos:
x_categorical = pd.get_dummies(x_categorical, columns=columns_categorical_train)

#Para valores numericos:
scaler = MinMaxScaler().fit(x_numerical)
x_scaled = scaler.transform(x_numerical)

x_categorical = pd.DataFrame(x_categorical, columns = x_categorical.columns)
x_scaled = pd.DataFrame(x_scaled, columns = x_numerical.columns)
#Juntamos los dos dataset escalados:
df_scaled_test = pd.concat( [x_categorical,x_scaled],axis=1)

#Eliminamos la columna 'Unnamed: 0'
df_scaled_test = df_scaled_test.drop('Unnamed: 0',axis=1)
#Por lo que nuestro dataset de entrenamiento final es:

df_scaled_test.columns

Index(['Utilities_AllPub', 'MSZoning_C (all)', 'MSZoning_FV', 'MSZoning_RH',
       'MSZoning_RL', 'MSZoning_RM', 'Street_Grvl', 'Street_Pave',
       'LotShape_IR1', 'LotShape_IR2',
       ...
       'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold'],
      dtype='object', length=254)

In [ ]:
df_scaled_test.to_csv('df_scaled_test.csv')
df_scaled_train.to_csv('df_scaled_train.csv')

Me acabo de dar cuenta que hay valores categoricos que están en train y que no están en test, voy a dropear todas aquellas columnas que no están en test al considerarlo como dato que no aporta información para este caso.

Hagamos este proceso de manera inteligente.

In [ ]:
df_scaled_train_columns = df_scaled_train.columns #Columnas texto train.
df_scaled_test_columns  = df_scaled_test.columns #Columnas texto test.

df_scaled_train_diff = df_scaled_train.drop(df_scaled_test_columns,axis=1) #Columnas que son distintas con datos
df_scaled_train = df_scaled_train.drop(df_scaled_train_diff.columns, axis =1) #Columnas iguales train y test

df_scaled_train.columns == df_scaled_test_columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

#Creemos modelos aprendiendo

Ahora mismo tenemos dos data set:

con df_scaled_train(tenemos X_train e y_train) tenemos que introducirlo en el algoritmo y 

con df_scaled_test(X_test) tenemos que comprobar como de eficiente es nuestro modelo

#XGBoost

In [ ]:
import xgboost
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

Buscamos los parametros optimos para en el entrenamiento y predicción:

In [ ]:
n_estimators=[100,500,900,1100,1500]
max_depth=[2,3,5,10,15]
booster =["gbtree","gblinear"]
base_score = [0.25,0.5,0.75,1]
learning_rate=[0.05,0.1,0.15,0.2]
min_child_weight=[1,2,3,4]

hyperparameter_grid = {
    'n_estimators':n_estimators,
    'max_depth':max_depth,
    'learning_rate':learning_rate,
    'min_child_weight':min_child_weight,
    'booster':booster,
    'base_score':base_score
}

regressor=xgboost.XGBRegressor()

random_cv = RandomizedSearchCV(estimator=regressor,
                               param_distributions=hyperparameter_grid,
                               cv=5,n_iter=50,
                               scoring='neg_mean_absolute_error',n_jobs=4,
                               verbose=5,
                               return_train_score=True,
                               random_state=42)

random_cv.fit(df_scaled_train,y_scaled)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[04:07:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


RandomizedSearchCV(cv=5, estimator=XGBRegressor(), n_iter=50, n_jobs=4,
                   param_distributions={'base_score': [0.25, 0.5, 0.75, 1],
                                        'booster': ['gbtree', 'gblinear'],
                                        'learning_rate': [0.05, 0.1, 0.15, 0.2],
                                        'max_depth': [2, 3, 5, 10, 15],
                                        'min_child_weight': [1, 2, 3, 4],
                                        'n_estimators': [100, 500, 900, 1100,
                                                         1500]},
                   random_state=42, return_train_score=True,
                   scoring='neg_mean_absolute_error', verbose=5)

Una vez entrenado, vemos cuales son los más optimos

In [ ]:
random_cv.best_estimator_

XGBRegressor(base_score=0.25, max_depth=2, n_estimators=900)

Los valores de arriba los pongo en el clasificador para predecir:

In [ ]:
classifier = xgboost.XGBRegressor(base_score=0.25, max_depth=2, n_estimators=900)

classifier.fit(df_scaled_train,y_scaled)

predictions = classifier.predict(df_scaled_test)
predictions

[04:20:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


array([0.0537113 , 0.19287859, 0.4231297 , ..., 0.05277383, 0.22915882,
       0.26483873], dtype=float32)

In [ ]:
predictions = predictions.reshape(-1, 1)
predictions_new_inverse = scalery.inverse_transform(predictions)
y_new_inverse = scalery.inverse_transform(y_scaled)

array([[185186.69],
       [196238.7 ],
       [214524.17],
       ...,
       [185112.25],
       [199119.9 ],
       [201953.44]], dtype=float32)

Guardamos el modelo

In [ ]:
import pickle

In [ ]:
filename = "xgboost_model.pkl"
pickle.dump(classifier, open(filename,"wb"))

Hacemos un csv con los registros. Del csv facilitado por Kaggle obtengo el ID.

In [ ]:
submision_df=pd.read_csv("sample_submission.csv")
predictions_df = pd.DataFrame(scalery.inverse_transform(predictions))
resultado= pd.concat([submision_df["Id"],predictions_df],axis=1)
resultado.columns=["Id","SalePrice"]
resultado.to_csv("sample_submissionXX.csv",index=False)